In [1]:
'''
Functions:
1. Compare LIS-CABLE with GRACE, GLEAM, & DOLCE
2. GW vs FD
3. plot time-series and spitial (difference) map
'''

'\nFunctions:\n1. Compare LIS-CABLE with GRACE, GLEAM, & DOLCE\n2. GW vs FD\n3. plot time-series and spitial (difference) map\n'

In [ ]:
from netCDF4 import Dataset
import netCDF4 as nc
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from scipy.interpolate import griddata
import matplotlib.ticker as mticker
from convert_units import get_land_var_scale, get_land_var_scale_offline
from common_utils import *

In [1]:
def calc_cable_tws(file_path, loc_lat, loc_lon, lat_name, lon_name):

    # calculate TWS for CABLE simulations
    # Input : 

    print("calc_cable_tws")

    Soil_thickness = [0.022, 0.058, 0.154, 0.409, 1.085, 2.872]
    off_file       = Dataset(file_path, mode='r')
    GWdz           = 5. #off_file.variables['GWdz'][:]
    SoilMoist      = off_file.variables['SoilMoist_inst'][:]


    time1, CanopInt= read_var(file_path, "CanopInt_inst", loc_lat, loc_lon, lat_name, lon_name)

    time2, GWMoist = read_var(file_path, "GWwb_tavg", loc_lat, loc_lon, lat_name, lon_name)
    GWMoist        = GWMoist*GWdz*1000.

    time3, SWE     = read_var(file_path, "SWE_inst", loc_lat, loc_lon, lat_name, lon_name)

    TWS            = GWMoist + CanopInt + SWE

    for i in np.arange(6):
        TWS = TWS + SoilMoist[:,i,:,:]*Soil_thickness[i]*1000
    
    year_2004       = datetime(2004,1,1)
    year_2009       = datetime(2009,12,31)

    TWS_0409 = spital_var(time1,TWS,year_2004,year_2009)
    
    TWS      = TWS - TWS_0409
    
    return time1,TWS


In [3]:
def read_var_to_daily_spatial_avg(file_paths, loc_lat, loc_lon, lat_var_name, lon_var_name):
    '''
    It is basically done
    '''
    
    # 
    time, rain = read_var_multi_file(file_paths, "Rainf_tavg", loc_lat, loc_lon, lat_var_name, lon_var_name)
    time, wtd  = read_var_multi_file(file_paths, "WaterTableD_tavg", loc_lat, loc_lon, lat_var_name, lon_var_name)  
    
    # ET
    time, evap = read_var_multi_file(file_paths, "Evap_tavg", loc_lat, loc_lon, lat_var_name, lon_var_name)    
    time, tveg = read_var_multi_file(file_paths, "TVeg_tavg", loc_lat, loc_lon, lat_var_name, lon_var_name)
    time, esoil= read_var_multi_file(file_paths, "ESoil_tavg", loc_lat, loc_lon, lat_var_name, lon_var_name)
    
    # run off 
    time, qs   = read_var_multi_file(file_paths, "Qs_tavg", loc_lat, loc_lon, lat_var_name, lon_var_name)
    time, qsb  = read_var_multi_file(file_paths, "Qsb_tavg", loc_lat, loc_lon, lat_var_name, lon_var_name)
    qs         = qs + qsb
    qsb        = None
    
    # sensible latent heat
    time, qh  = read_var_multi_file(file_paths, "Qh_tavg", loc_lat, loc_lon, lat_var_name, lon_var_name)
    time, qle  = read_var_multi_file(file_paths, "Qle_tavg", loc_lat, loc_lon, lat_var_name, lon_var_name)

    
    ## CAUTION: 6 layers
    time, sm   = read_var_multi_file(file_paths, "SoilMoist_inst", loc_lat, loc_lon, lat_var_name, lon_var_name) 
    time, st   = read_var_multi_file(file_paths, "SoilTemp_inst", loc_lat, loc_lon, lat_var_name, lon_var_name) 

    # translate to daily data
    rain_daily = time_clip_to_day(time, rain, time_s, time_e)
    wtd_daily  = time_clip_to_day(time, wtd, time_s, time_e)
    
    evap_daily = time_clip_to_day(time, evap, time_s, time_e)
    tveg_daily = time_clip_to_day(time, tveg, time_s, time_e)
    esoil_daily= time_clip_to_day(time, esoil, time_s, time_e)
    
    qs_daily   = time_clip_to_day(time, qs, time_s, time_e)
    
    qh_daily   = time_clip_to_day(time, qh, time_s, time_e)
    qle_daily  = time_clip_to_day(time, qle, time_s, time_e)
    
    sm_daily   = time_clip_to_day(time, sm, time_s, time_e)
    st_daily   = time_clip_to_day(time, st, time_s, time_e)
    
    # spitial average
    rain_1D = time_series_var(time,rain_daily,time_s,time_e)
    wtd_1D  = time_series_var(time,wtd_daily,time_s,time_e)
    
    evap_1D = time_series_var(time,evap_daily,time_s,time_e)
    tveg_1D = time_series_var(time,tveg_daily,time_s,time_e)
    esoil_1D= time_series_var(time,esoil_daily,time_s,time_e)
    
    qs_1D   = time_series_var(time,qs_daily,time_s,time_e)
    
    qh_1D   = time_series_var(time,qh_daily,time_s,time_e)
    qle_1D  = time_series_var(time,qle_daily,time_s,time_e)
    
    sm_2D   = time_series_var(time,sm_daily,time_s,time_e)
    st_2D  = time_series_var(time,st_daily,time_s,time_e)
    
    return rain_1D, wtd_1D, evap_1D, tveg_1D, esoil_1D, qs_1D, qh_1D, qle_1D, sm_2D, st_2D



In [ ]:
def read_DOLCE_GLEAM(file_paths, var_names, year_s, year_e, loc_lat=None, loc_lon=None, lat_names=None, lon_names=None, message=None):

    print("======== In plot_spital_map =========")

    # Open the NetCDF4 file (add a directory path if necessary) for reading:
    time1, Var1  = read_var(file_paths[0], var_names[0], loc_lat, loc_lon, lat_names[0], lon_names[0])
    time1, lats1 = read_var(file_paths[0], lat_names[0], loc_lat, loc_lon, lat_names[0], lon_names[0])
    time1, lons1 = read_var(file_paths[0], lon_names[0], loc_lat, loc_lon, lat_names[0], lon_names[0])
    scale        = get_scale(var_names[0])
    var1         = spital_var(time1,Var1,year_s,year_e)*scale

    if len(file_paths) > 1:
        time2, Var2  = read_var(file_paths[1], var_names[1], loc_lat, loc_lon, lat_names[1], lon_names[1])
        time2, lats2 = read_var(file_paths[1], lat_names[1], loc_lat, loc_lon, lat_names[1], lon_names[1])
        time2, lons2 = read_var(file_paths[1], lon_names[1], loc_lat, loc_lon, lat_names[1], lon_names[1])
        scale        = get_scale(var_names[1])
        var2         = spital_var(time2,Var2,year_s,year_e)*scale
        # var1_regrid  = griddata((lats1.flatten(),lons1.flatten()), var1.flatten(), (lats2, lons2), method='nearest') # 'linear' 'cubic'
        var          = var2 - var1
    else:
        var          = var1

    # np.savetxt("test_var.txt",var,delimiter=",")

    fig = plt.figure(figsize=(6,5))
    ax = plt.axes(projection=ccrs.PlateCarree())

    if loc_lat == None:
        ax.set_extent([140,154,-40,-28])
    else:
        ax.set_extent([loc_lon[0],loc_lon[1],loc_lat[0],loc_lat[1]])

    ax.coastlines(resolution="50m",linewidth=1)
    # Add gridlines
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,linewidth=1, color='black', linestyle='--')
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xlines = True

    if loc_lat == None:
        gl.xlocator = mticker.FixedLocator([140,145,150])
        gl.ylocator = mticker.FixedLocator([-40,-35,-30])
    else:
        gl.xlocator = mticker.FixedLocator(loc_lon)
        gl.ylocator = mticker.FixedLocator(loc_lat)

    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    gl.xlabel_style = {'size':10, 'color':'black'}
    gl.ylabel_style = {'size':10, 'color':'black'}
    # Plot windspeed

    # clevs = np.linspace( 0.,1500., num=31)
    plt.contourf(lons1, lats1, var,  transform=ccrs.PlateCarree(), extend='both',cmap=plt.cm.BrBG) # clevs,
    plt.title(var_names[0], size=16)
    cb = plt.colorbar(ax=ax, orientation="vertical", pad=0.02, aspect=16, shrink=0.8)
    # cb.set_label(units,size=14,rotation=270,labelpad=15)
    cb.ax.tick_params(labelsize=10)
    if message == None:
        message = var_names[0]
    else:
        message = message + "_" + var_names[0]
    plt.savefig('./plots/19Oct/spatial_map_obs_'+message+'.png',dpi=300)



In [ ]:
def plot_time_series( file_paths, var_names, year_s, year_e, loc_lat=None, loc_lon=None,
                      lat_names=None, lon_names=None, message=None ):

    print("======== In plot_time_series =========")

    fig, ax = plt.subplots()

    # plot line 1
    if var_names[0] == "TWS":
        Time1, Var1 = calc_cable_tws(file_paths[0], loc_lat, loc_lon, lat_names[0], lon_names[0])
        time1, var1 = time_series_var(Time1, Var1, year_s, year_e)
    elif var_names[0] == "lwe_thickness":
        Time1, Var1 = read_var(file_paths[0], var_names[0], loc_lat, loc_lon, lat_names[0], lon_names[0])
        time1, var1 = time_series_var(Time1, Var1*10., year_s, year_e)
    else:
        Time1, Var1 = read_var(file_paths[0], var_names[0], loc_lat, loc_lon, lat_names[0], lon_names[0])
        time1, var1 = time_series_var(Time1, Var1, year_s, year_e) # np.cumsum()

    t1 = []
    for i in np.arange(len(time1)):
        t1.append(time1[i].days)
    if var_names[0] in ['Qs','Qsb','Rainf','Evap','ESoil','ECanop','TVeg']:
        scale = 24.*3600.
    else:
        scale = 1.
    print("var1*scale")
    print(var1*scale)
    ax.plot(t1, var1*scale, c = "blue", label="GW", alpha=0.5)


    # plot line 2
    if len(file_paths) > 1:
        if var_names[1] == "TWS":
            Time2, Var2 = calc_cable_tws(file_paths[1], loc_lat, loc_lon, lat_names[1], lon_names[1])
            time2, var2 = time_series_var(Time2, Var2, year_s, year_e)
        elif var_names[1] == "lwe_thickness":
            Time2, Var2 = read_var(file_paths[1], var_names[1], loc_lat, loc_lon, lat_names[1], lon_names[1])
            time2, var2 = time_series_var(Time2, Var2*10., year_s, year_e)
        else:
            Time2, Var2 = read_var(file_paths[1], var_names[1], loc_lat, loc_lon, lat_names[1], lon_names[1])
            time2, var2 = time_series_var(Time2, Var2, year_s, year_e) #np.cumsum()
        t2 = []
        for i in np.arange(len(time2)):
            t2.append(time2[i].days)
        if var_names[1] in ['Qs','Qsb','Rainf','Evap','ESoil','ECanop','TVeg']:
            scale = 24.*3600.
        else:
            scale = 1.

        print("var2*scale")
        print(var2*scale)
        ax.plot(t2, var2*scale, c = "green", label="FD", alpha=0.5)

    # plot line 3
    if len(file_paths) > 2:
        if var_names[2] == "TWS":
            Time3, Var3 = calc_cable_tws(file_paths[2], loc_lat, loc_lon, lat_names[2], lon_names[2])
            time3, var3 = time_series_var(Time3, Var3, year_s, year_e)
        elif var_names[2] == "lwe_thickness":
            Time3, Var3 = read_var(file_paths[2], var_names[2], loc_lat, loc_lon, lat_names[2], lon_names[2])
            time3, var3 = time_series_var(Time3, Var3*10., year_s, year_e)
        else:
            Time3, Var3 = read_var(file_paths[2], var_names[2], loc_lat, loc_lon, lat_names[2], lon_names[2])
            time3, var3 = time_series_var(Time3, Var3, year_s, year_e) # np.cumsum()
        t3 = []
        for i in np.arange(len(time3)):
            t3.append(time3[i].days)
        if var_names[2] in ['Qs','Qsb','Rainf','Evap','ESoil','ECanop','TVeg']:
            scale = 24.*3600.
        else:
            scale = 1.

        print("var3*scale")
        print(var3*scale)
        ax.plot(t3, var3*scale, c = "red", label="FD", alpha=0.5)

    # ax.set_xlim([np.min(var1*scale,var2*scale), np.max(var1*scale,var2*scale)])
    # Time2, Var2 = read_var(file_paths[1], var_names[1], loc_lat, loc_lon, lat_name[1], lon_name[1])
    # time2, var2 = time_series_var(Time2,Var2,year_s,year_e)
    # var = np.zeros((2,len(var1)))
    # var[0,:] = var1
    # var[1,:] = var2
    # ax.plot(t1, var*scale, alpha=0.5)
    # ax.set_ylabel('mm')
    ax.set_title(var_names[0])
    # ax.set_xticks(x1[::1440])
    # ax.set_xticklabels(np.arange(year_s,year_e,1))
    ax.legend()

    fig.tight_layout()
    if message == None:
        message = var_names[0]
    else:
        message = message + "_" + var_names[0]
    if loc_lat != None:
        message = message + "_lat="+str(loc_lat[0]) +"-"+str(loc_lat[1]) + "_lon="+str(loc_lon[0])+"-"+str(loc_lon[1])

    plt.savefig('./plots/19Oct/time_series_lis_vs_off_'+message+'.png',dpi=300)



In [ ]:
if __name__ == "__main__":

    # #######################
    #     path setting      #
    # #######################

    DOLCE_path = "/g/data/w97/mm3972/data/DOLCE/v3/"
    DOLCE_file = DOLCE_path+"DOLCE_v3_2000-2018.nc"
    GLEAM_path = "/g/data/w97/W35_GDATA_MOVED/Shared_data/Observations/Global_ET_products/GLEAM_v3_5/v3-5a/monthly/"
    GLEAM_file = GLEAM_path + "E_1980-2020_GLEAM_v3.5a_MO.nc"
    GRACE_path = "/g/data/w97/mm3972/data/GRACE/GRACE_JPL_RL06/GRACE_JPLRL06M_MASCON/"
    GRACE_file = GRACE_path + "GRCTellus.JPL.200204_202004.GLO.RL06M.MSCNv02CRI.nc"
    CABLE_path = 
    CABLE_file = 

    # #######################
    #        variable       #
    # #######################

    var_of_interest = [ ["GWMoist","GWMoist"],
                        ["Evap","Evap"],
                        ["TVeg","TVeg"],
                        ["ESoil","ESoil"],
                        ["ECanop","ECanop"],
                        ["Qs","Qs"],
                        ["Qsb","Qsb"],
                        ["Rainf","Rainf"],
                        ["WatTable","WatTable"],
                        ["Qle","Qle"],
                        ["Qh","Qh"],
                        ["Qg","Qg"],
                        ["RadT","RadT"],
                        ["VegT","VegT"],
                        ["Fwsoil","Fwsoil"]]



    ####################################
    #         plot_time_series         #
    ####################################

    year_s       = datetime(2000,1,1)
    year_e       = datetime(2019,12,31)
    loc_lat      = [-40,-28]
    loc_lon      = [140,154]

    # var_names       = [ ["GWMoist","GWMoist"],
    #                     ["Evap","Evap"],
    #                     ["TVeg","TVeg"],
    #                     ["ESoil","ESoil"],
    #                     ["ECanop","ECanop"],
    #                     ["Qs","Qs"],
    #                     ["Qsb","Qsb"],
    #                     ["WatTable","WatTable"],
    #                     ["Qle","Qle"],
    #                     ["Qh","Qh"],
    #                     ["Qg","Qg"],
    #                     ["RadT","RadT"],
    #                     ["VegT","VegT"],
    #                     ["Fwsoil","Fwsoil"]]

    # file_paths  = [GW_off_file, FD_off_file]
    
    # for var_name in var_names:
    #     lat_names = ["latitude","latitude"]
    #     lon_names = ["longitude","longitude"]
    #     message   = "GW_vs_FD"
    #     plot_time_series(file_paths, var_name, year_s, year_e, loc_lat=loc_lat, loc_lon=loc_lon,
    #                      lat_names=lat_names, lon_names=lon_names, message=message)

    # file_paths= [GRACE_file, GW_off_file, FD_off_file]
    # var_names = ["lwe_thickness","TWS","TWS"]

    # GLEAM_vs_CABLE
    file_paths= [GLEAM_file, GW_off_file, FD_off_file]
    var_names = ["E","Evap","Evap"]    
    lat_names = ["lat","latitude","latitude"]
    lon_names = ["lon","longitude","longitude"]
    message   = "GLEAM_vs_CABLE"
    plot_time_series(file_paths, var_names, year_s, year_e, loc_lat=loc_lat, loc_lon=loc_lon,
                     lat_names=lat_names, lon_names=lon_names, message=message)

    # DOLCE_vs_CABLE
    # file_paths= [DOLCE_file, GW_off_file, FD_off_file]
    # var_names = ["hfls","Qle","Qle"]    
    # lat_names = ["lat","latitude","latitude"]
    # lon_names = ["lon","longitude","longitude"]
    # message   = "DOLCE_vs_CABLE"
    # plot_time_series(file_paths, var_names, year_s, year_e, loc_lat=loc_lat, loc_lon=loc_lon,
    #                  lat_names=lat_names, lon_names=lon_names, message=message)